In [1]:
import numpy as np
from pprint import pprint
from itertools import permutations

In [2]:
LIMIT_DISTANCE = 10  #см. Это диаметр маркера
EPS = 0.5 #см. Насколько точно рамка определяет положение ножек маркера?

In [3]:
#Инициализируем маркеры. Их начальное положение. Угол поворота = 0
target_car = [[2, 8], [7, 6], [9, 13]]
target_tree = [[12, 8], [17, 5], [19, 13]]

### класс Точки-вектора

точка с координатами x, y

- __add__ сложение векторов


- __sub__ вычитание


- __repr__ вывод



In [4]:
class Vector:
    def __init__(self, *args, **kwargs):
        if len(args) == 0:
            self.x = kwargs.get('x', 0.0)
            self.y = kwargs.get('y', 0.0)
        else:
            self.x = args[0]
            self.y = args[1]

    # function to add two vectors
    def __add__(self, other):
        return Vector(x=self.x+other.x,y=self.y+other.y)

    # function to subtract vectors
    def __sub__(self, other):
        return Vector(x=self.x-other.x,y=self.y-other.y)

    # make the vector printable
    def __repr__(self):
        return "x: %s y: %s" % (self.x, self.y)

In [5]:
puk = Vector(1,2)

In [7]:
puk - Vector(2, 4)

x: -1 y: -2

### вспомогательные функции для работы с геометрией

- distance(p1, p2) вычисление длины по двум точкам


- calc_clockwise_angle(p1, p2) расчет угла по часовой стрелке


- rotate_vector (vec, angle) вращение вектора


- clockwise_difference_between_angles(original_angle, changed_angle) вычисление разницы между углами


- is_point_in_circle(centerX, centerY, radius, pointX, pointY, aspectCorrectionFactor) проверка попадания ОДНОЙ ТОЧКИ в окружность радиуса радиус с центром в указанных точках

In [8]:
# function to calculate the distance between two points
def distance(p1, p2):
    dist = np.sqrt( ((p2.x - p1.x)**2) + ((p2.y - p1.y)**2))
    return dist

# function to calculate the clockwise angle between 12 o'clock (on screen)coming out of p1 and p1-p2
def calc_clockwise_angle(p1, p2):
    origin = p1
    a = Vector(p1.x, -500.0)
    b = p2
    v1 = a - origin
    v2 = b - origin
    angle = -math.atan2(v2.x*v1.y - v2.y*v1.x, v2.x*v1.x+v2.y*v1.y)

    if (angle < 0):
        angle += 2*math.pi
    return angle

# function to rotate a vector by a certain angle
def rotate_vector(vec, angle):
    rotated_vector = Vector()
    cos_angle = math.cos(angle)
    sin_angle = math.sin(angle)
    rotated_vector.x = vec.x*cos_angle - vec.y*sin_angle
    rotated_vector.y = vec.x*sin_angle + vec.y*cos_angle
    return rotated_vector

# function to calculate the clockwise difference between two angles
def clockwise_difference_between_angles(original_angle, changed_angle):
    if original_angle > changed_angle:
        changed_angle += 2*math.pi
    angle = changed_angle - original_angle
    return angle

# function to check if a point lies in a circle 
def is_point_in_circle(centerX, centerY, radius, pointX, pointY, aspectCorrectionFactor):
    distance_from_center = np.sqrt(((centerX - pointX)* aspectCorrectionFactor) ** 2  + (centerY - pointY) ** 2)
    return distance_from_center <= radius

### класс треугольник

set_cursors получает три точки устанавоивает в правильном порядке вершины стороны и ориентацию треугольника

arrange_sides ранжирует стороны по длине

get_IDS

pretty_print выводит инфу о треугольнике

In [9]:
class Triangle:
   
    # The side between vertices 1 and 2 has to be the shortest side
    # The side between vertices 2 and 3 has to be he intermediate side
    # The side between vertices 3 and 1 has to be the longest side

    clockwise_rotation = True

    # function calculate the side lengths and orientation of the trinagle
    def set_cursors(self, p1, p2, p3):
        points_sides = arrange_sides(p1, p2, p3)
        self.v1 = points_sides['v1']
        self.v2 = points_sides['v2']
        self.v3 = points_sides['v3']
        self.long_side = points_sides['a']
        self.middle_side = points_sides['b']
        self.short_side = points_sides['c']
        
        # check for rotation order (v2 left or right of v1 to v3)
        if ((self.v3.x - self.v1.x)*(self.v2.y - self.v1.y) - (self.v3.y - self.v1.y)*(self.v2.x - self.v1.x)) > 0:
            self.clockwise_rotation = True
        else:
            self.clockwise_rotation = False
                
    def arrange_sides(self, p1, p2, p3):
        
        # get the length of the triangle sides and put them in order
        a = distance(p1, p2) + 0.000000001 # prevent same size sides
        b = distance(p2, p3)
        c = distance(p3, p1) - 0.000000001 # prevent same size sides
        
        # return [v1, v2, v3, long_side, middle_side, short_side]
        
        if (a > b) & (a > c):
            # a is longest
            if b > c:
                # b is middle
                # c is shortest
                return {'v1':p1, 'v2':p3, 'v3':p2, 'a':a, 'b':b, 'c':c}
            else:
                # c is middle
                # b is shortest
                return {'v1':p2, 'v2':p3, 'v3':p1, 'a':a, 'b':c, 'c':b} #[p2, p3, p1, a, c, b]

        elif (b > a) & (b > c):
            # b is longest
            if a > c:
                # a is middle
                # c is shortest
                return {'v1':p3, 'v2':p2, 'v3':p2, 'a':b, 'b':a, 'c':c} #[p3, p1, p2, b, a, c]
            else:
                # c is middle
                # a is shortest
                return {'v1':p2, 'v2':p1, 'v3':p3, 'a':b, 'b':c, 'c':a} #[p2, p1, p3, b, c, a]
                
        elif (c > a) & (c > b):
            # c is longest
            if a > b:
                # a is middle
                # b is shortest
                return {'v1':p3, 'v2':p2, 'v3':p1, 'a':c, 'b':a, 'c':b} #[p3, p2, p1, c, a, b]
                
            else:
                # b is middle
                # a is shortest
                return {'v1':p1, 'v2':p2, 'v3':p3, 'a':c, 'b':b, 'c':a}
                

    # function to return the IDs of the cursor points building the triangle
    def get_IDs(self):
        id_list = [self.v1.id, self.v2.id, self.v3.id]
        return id_list

    # function to print a fancy info about the triangle to console
    def pretty_print(self):
        print('\n--------Triangle -------------------\n')
        print('Ids are:', self.get_IDs())
        print(self.v1)
        print(self.v2)
        print(self.v3)
        print ('Longest side v3-v1:', self.long_side)
        print ('Middle side v2-v3:', self.middle_side)
        print ('Shortest side v1-v2:', self.short_side)
        if self.clockwise_rotation:
            print ('Middle side is counter clockwise oriented')
        else:
            print ('Middle side is clockwise oriented')

### создание всевозможных треугольников из списка точек

получаем список точек с рамки [(x, y), ...]

составляем из них всевозможные треугольники 

возвращаем список треугольников

In [10]:
# function to create all possible triangles from a list of points 
# создание всевозможных треугольников из списка точек

def create_triangles_from_cursors(*args):
    
    list_of_cursors = args[0]
    triangles = []
    
    # create  tuples with the length of 3 points, in sorted order, no repeated elements
    for combination in list(itertools.combinations(list_of_cursors, 3)):
        # create triangles from the tuples
        t = triangle()
        t.set_cursors(combination[0], combination[1], combination[2])
        triangles.append(t)
    return triangles

### сравнение двух треугольников с учетом погрешности
получаем два треугольника

если равны то true;

если нет то false;

In [11]:
# function to compare two triangles on their similarity
# сравнение двух треугольников с учетом погрешности

def compare_triangles(t1, t2, tolerance):
    
    # check if both triangles have the same rotation
    if t1.clockwise_rotation == t2.clockwise_rotation:
    
        # check if the difference of the long sides is within tolerance
        long_side_difference = math.fabs(t1.long_side - t2.long_side)
        if long_side_difference <= tolerance:

            # check if the difference of the middle sides is within tolerance
            middle_side_difference = math.fabs(t1.middle_side - t2.middle_side)
            if middle_side_difference <= tolerance:
            
                # check if the difference of the short sides is within tolerance
                short_side_difference = math.fabs(t1.short_side - t2.short_side)
                if short_side_difference <= tolerance:
                    
                    # triangles are considered equal
                    return True
    else:
        # triangles are considered different
        return False

### класс точек из которых состоит маркер

In [12]:
# class for the cursors a tangible consists of
class MarkerCursor():
    
    # declare variables
    last_known_positionX = 0.0
    last_known_positionY = 0.0
    last_live_ID = 0
    offset_from_centerX = 0.0
    offset_from_centerY = 0.0
    votes = 0

    # function to make the object writable to disk
    def __getstate__(self):
        return self.__dict__
    
    # function to make the object loadable from disk
    def __setstate__(self, d):
        self.__dict__.update(d)

    # function to initalize the marker point with values from the arguments
    def __init__(self, *args, **kwargs):
        self.id = kwargs.get('id', 0)
        self.x = kwargs.get('x', 0.0)
        self.y = kwargs.get('y', 0.0)
    
    # function calculating the poinst offset from the calibration center
    def calc_offset(self, calibrationCenter):
        self.offset_from_centerX = self.x - calibrationCenter.x
        self.offset_from_centerY = self.y - calibrationCenter.y
    
    # function to increase the votes to a point by one
    def vote(self):
        self.votes = self.votes + 1
        
    # function to return the points votes
    def get_votes(self):
        return self.votes
        
    # function to dump the points votes
    def delete_votes(self):
        self.votes = 0
    
    # function returning the last live cursor id associated with this tangible cursor
    def get_last_live_ID(self):
        return self.last_live_ID
    
    # function to print the points votes, position and offset from center
    def pretty_print(self):
        print ('ID:', self.id, 'Votes:', self.votes)
        print('Position x:', self.x, 'y:', self.y, 'Offset from center', self.offset_from_centerX, self.offset_from_centerY)

### класс Маркер

- register_cursors регистрация точек в центре рамки как маркер

In [ ]:
# class defining a tangible
class Marker():
    
    # function called when object is created
    def __init__(self):
        
        self.id = 0
        self.position = Vector()
        self.rotation = 0.0
        self.outOfJailCard = 2
        self.marker_cursors = {}
        self.marker_triangles = []
        self.currently_recognized = False
        self.last_succesful_run_IDs = []
        self.external_ID_to_marker_cursor_ID_dict = {}
        self.external_ID_to_marker_cursor_ID_dict_reverse = {}
        
        self.marker_position_filter = positionFilter()
        self.marker_rotation_filter = rotationFilter()
    
    # function to set the marker up with the cursors in the argument
    def register_cursors(self, *args,**kwargs):
        self.id = kwargs.get('id', 0)
        cursors = args[0]
        
        # set the center for tangible creation, if not provdided otherwise
        calibration_center = Vector(settings.get('calibrationCenter')[0], settings.get('calibrationCenter')[1])
        
        # add points to dictionary
        # for every point register it as marker cursor and calc offset
        for c in cursors:
            self.marker_cursors[c.id] = MarkerCursor(x=c.x, y=c.y)
            self.marker_cursors[c.id].calc_offset(calibration_center)
        # create triangles from points
        self.marker_triangles = create_triangles_from_cursors(cursors)

    # function to vote for points with a IDs in argument
    def vote_for_IDs(self, arg):
        for id in arg:
            self.tangible_cursors[id].vote()
    
    # function to delete the votes of every marker point
    def drop_votes(self):
        for id in self.tangible_cursors.keys():
            self.tangible_cursors[id].delete_votes()
    
    # function to retrieve the marker cursor id that belongs to a live cursor 
    def external_ID_to_marker_cursor_ID(self, external_ID):
        if external_ID in self.external_ID_to_tangible_cursor_ID_dict:
            tangible_cursor_ID = self.external_ID_to_marker_cursor_ID_dict[external_ID]
            return tangible_cursor_ID
        else:
            return []
    
    # function to link live cursors with marker cursors
    def combine_external_ID_with_marker_cursor_ID(self, external_ID, marker_cursor_ID):
        try:
            # if one entry(exID) already has this entry (curID)
            if marker_cursor_ID in self.external_ID_to_marker_cursor_ID_dict_reverse:
                # delete this entry
                # get the key the normal dictionary has
                external_ID_to_delete = self.external_ID_to_marker_cursor_ID_dict_reverse[tangible_cursor_ID]
                # delete this entry
                del self.external_ID_to_marker_cursor_ID_dict[external_ID_to_delete]
                # delete also in the inverse dict
                del self.external_ID_to_marker_cursor_ID_dict_reverse[marker_cursor_ID]

            if external_ID in self.external_ID_to_marker_cursor_ID_dict:
                marker_cursor_ID_to_delete = self.external_ID_to_marker_cursor_ID_dict[external_ID]
                del self.external_ID_to_marker_cursor_ID_dict_reverse[marker_cursor_ID_to_delete]
                del self.external_ID_to_marker_cursor_ID_dict[external_ID]

            self.external_ID_to_marker_cursor_ID_dict[external_ID] = marker_cursor_ID
            self.external_ID_to_marker_cursor_ID_dict_reverse[marker_cursor_ID] = external_ID

        except Exception, e:
            print("Failure: %s" % e)
            print('Error in combineExternalIdWithTangibleCursorID')

            
    # function to retrieve the last live id belonging to marker cursor
    def marker_cursor_ID_to_external_ID(self, marker_cursor_ID):
        if marker_cursor_ID in self.external_ID_to_marker_cursor_ID_dict_reverse:
            external_ID = self.external_ID_to_marker_cursor_ID_dict_reverse[marker_cursor_ID]
            return external_ID
        else:
            return []
        
    def set_position(self, idx, vertex):
        self.marker_cursors[idx].last_known_positionX = vertex.x
        self.marker_cursors[idx].last_known_positionY = vertex.y
        self.combine_external_ID_with_marker_cursor_ID(vertex.id, idx)
        self.marker_cursors[idx].last_live_ID = vertex.id
        
    # function to set update the last position where a set of tangible cursors has been seen
    def set_last_known_positions(self, ids, triangle):
        self.set_position(ids[0], triangle.v1)
        self.set_position(ids[1], triangle.v2)
        self.set_position(ids[2], triangle.v3)
        
    
    # function to compare momentary triangles with the triangles defining the pattern
    def compare_marker_triangles_and_external_triangles(self, *args):
        self.external_cursors = args[0]
        # create all possible triangles from received cursors
        external_triangles = create_triangles_from_cursors(self.external_cursors)
        
        tolerance = settings.get('tolerance')
        
        for external_triangle in external_triangles:
            for internal_triangle in self.marker_triangles:
        
                if compare_triangles(external_triangle, internal_triangle, tolerance):
                    # set last known position of tangible cursors and vote for them
                    ids = internal_triangle.get_IDs()
                    self.set_last_known_positions(ids, external_triangle)
                    self.voteForIDs(ids)
        
        # sort points by their received votes
        sorted_keys = []
        for key, value in sorted(self.marker_cursors.iteritems(), key=lambda (k,v): (v.votes,k), reverse=True):
            sorted_keys.append(key)
        

        # check if their have been enough votes
        highest_votes = self.marker_cursors[sorted_keys[1]].votes
        needed_votes = settings.get('neededVotes')
        if settings.get('debugVotes'):
            print("Highest", highest_votes)
            print("Needed", needed_votes)
        
        if highest_votes >= needed_votes:
            self.currently_recognized = True
            self.outOfJailCard = settings.get('outOfJailCard')
            
            self.last_succesful_run_IDs = []
            for marker_cursor_ID in sorted_keys:
                one_successful = self.marker_cursor_ID_to_external_ID(marker_cursor_ID)
                if one_successful != []:
                    self.last_succesful_run_IDs.append(one_successful)
            
            # calculate marker postion and rotation with best recognized cursors
            id1 = sorted_keys[0]
            id2 = sorted_keys[1]
            self.calculate_marker_position_and_rotation(id1, id2)
            
        else:
            #compare current live IDs with last succeful recognized IDs
            live_IDs = []
            for external_cursor in self.external_cursors:
                live_IDs.append(external_cursor.id)
            cursor_intersection = filter(set(live_IDs).__contains__, self.last_succesful_run_IDs)
            
            #check if at least two cursors are left
            if len(cursor_intersection) > 1:
                self.currently_recognized = True
                self.outOfJailCard = settings.get('outOfJailCard')
                
                # take first two live ids and calculate with them
                id1 = cursor_intersection[0]
                id2 = cursor_intersection[1]
                self.calculate_marker_position_and_rotation_wth_live_IDs(id1, id2)
                
            else:
                # to prevent single "glitches" from disturbing the signal the tangible has some out of jail cards
                if self.outOfJailCard > 0:
                    self.outOfJailCard -= 1
                    self.currently_recognized = True
                else:
                    self.currently_recognized = False
                    if settings.get('debugVotes'):
                        print ('No match for ID:', self.id)

        self.drop_votes()


    # function to calculate the current position and rotation out of two known tangible cursors
    def calculate_marker_position_and_rotation_with_live_IDs(self, id1, id2) :
        
        #translate from live ID to internal ID
        internal_cursor_ID1 = self.external_ID_to_marker_cursor_ID(id1)
        internal_cursor_ID2 = self.external_ID_to_marker_cursor_ID(id2)
        
        # create dictionary with live cursors

        live_cursors = {}
        for c in self.external_cursors:
            live_cursors[c.id] = c

        # calculate original rotation angle
        p1_old = Vector(self.marker_cursors[internalCursorID1].offset_from_centerX, self.marker_cursors[internal_cursor_ID1].offset_from_centerY)
        p2_old = Vector(self.marker_cursors[internalCursorID2].offset_from_centerX, self.marker_cursors[internal_cursor_ID2].offset_from_centerY)
        rotation_angle_in_centered_marker = calc_clockwise_angle(p1_old, p2_old)
        
        # calculate the current angle
        
        p1_now = Vector(live_cursors[id1].x, live_cursors[id1].y)
        p2_now = Vector(live_cursors[id2].x, live_cursors[id2].y)
        rotation_angle_of_marker_now = calc_clockwise_angle(p1_now, p2_now)   
        
        # calculate the difference between the two angles
        current_rotation = clockwise_difference_between_angles(rotation_angle_in_centered_marker, rotation_angle_of_marker_now); 
      
    
  ####################################################################  
    
        # check if the rotation filter is set to pre
        if settings.get('rotationFilterPosition') == 'pre':
            # add current rotation value to the rotation filter
            self.tangible_rotation_filter.addValue(currentRotation)
            # get rotation value from filter 
            currentRotation = self.tangibleRotationFilter.getState()
        
        # calculate the vector form current p1 to the tangible center
        shiftOfId1 = rotateVector(p1old, currentRotation)
        # calculate position
        currentPosition = p1now - shiftOfId1
        
        # check if the position filter is active
        if settings.get('positionFilterActive'):
			# add current position to filter
			self.tangiblePositionFilter.addXvalue(currentPosition.x)
			self.tangiblePositionFilter.addYvalue(currentPosition.y)
			# get position from filter
			currentPosition.x = self.tangiblePositionFilter.getXstate()
			currentPosition.y = self.tangiblePositionFilter.getYstate()
						
		# check if post rotation filter is active
		if settings.get('rotationFilterPosition') == 'post':
			# add current rotation value to the rotation filter
			self.tangibleRotationFilter.addValue(currentRotation)
			# get rotation value from filter 
			currentRotation = self.tangibleRotationFilter.getState()
		
		# set position and rotation
		self.position = currentPosition
		self.rotation = currentRotation	
				
            ###############################################################
            
            
    # function to calculate the current position and rotation out of two known tangible cursors
    def calculate_marker_position_and_rotation(self, id1, id2) :
        
        # calculate original rotation angle
        p1_old = Vector(self.marker_cursors[id1].offset_from_centerX, self.marker_cursors[id1].offset_from_centerY)
        p2_old = Vector(self.marker_cursors[id2].offset_from_centerX, self.marker_cursors[id2].offset_from_centerY)
        rotation_angle_in_centered_marker = calc_clockwise_angle(p1_old,p2_old)
        
        # calculate the current angle
        p1_now = Vector(self.marker_cursors[id1].last_known_positionX, self.marker_cursors[id1].last_known_positionY)
        p2_now = Vector(self.marker_cursors[id2].last_known_positionX, self.marker_cursors[id2].last_known_positionY)
        rotation_angle_of_marker_now = calc_clockwise_angle(p1_now, p2_now)  
        
        # calculate the difference between the two angles
        current_rotation = clockwise_difference_between_angles(rotation_angle_in_centered_marker, rotation_angle_of_marker_now) 
        
        
        ############################################################################
        # check if the rotation filter is set to pre
        if settings.get('rotationFilterPosition') == 'pre':
            # add current rotation value to the rotation filter
            self.tangibleRotationFilter.addValue(currentRotation)
            # get rotation value from filter 
            currentRotation = self.tangibleRotationFilter.getState()
            
        # calculate the vector form current p1 to the tangible center
        shiftOfId1 = rotateVector(p1old, currentRotation)
        # calculate position
        currentPosition = p1now - shiftOfId1
        
        # check if the position filter is active
        if settings.get('positionFilterActive'):
            # add current position to filter
            self.tangiblePositionFilter.addXvalue(currentPosition.x)
            self.tangiblePositionFilter.addYvalue(currentPosition.y)
            # get position from filter
            currentPosition.x = self.tangiblePositionFilter.getXstate()
            currentPosition.y = self.tangiblePositionFilter.getYstate()

        # check if post rotation filter is active
        if settings.get('rotationFilterPosition') == 'post':
            # add current rotation value to the rotation filter
            self.tangibleRotationFilter.addValue(currentRotation)
            # get rotation value from filter 
            currentRotation = self.tangibleRotationFilter.getState()
            
            #####################################################################
            
        # set position and rotation
        self.position = current_position
        self.rotation = current_rotation
    
    # function returning a list of all live ids that have been lately identified to belong to the tangible
    def get_live_cursor_IDs(self):
        cursors = []
        for c in self.marker_cursors:
            id = self.marker_cursors[c].get_last_live_ID()
            cursors.append(id)
        return cursors 
    
    # function to make the tangible savable to disk
    def __getstate__(self):
        try:
            list = []
            list.append(self.__dict__)
            list.append(self.marker_cursors)
            list.append(self.external_ID_to_marker_cursor_ID_dict)
            list.append(self.external_ID_to_marker_cursor_ID_dict_reverse)
            return list

        except Exception, e:
            print ("Failure: %s" % e)
            print ("in function tangible getstate")
    
    # function to make the tangbile loadable from disk
    def __setstate__(self, d):
        try:
            self.__dict__.update(d[0])
            self.marker_cursors = d[1]
            self.external_ID_to_marker_cursor_ID_dict = d[2]
            self.external_ID_to_marker_cursor_ID_dict_reverse = d[3]
    
        except Exception, e:
            print ("Failure: %s" % e)
    print ("in function tangible setstate)

### Класс хранения маркеров

In [ ]:
# class to store the tangible objects
class MarkerContainer():

    # function is called when the container is created
    def __init__(self):
        self.dict_of_markers = {}

    # function to check if tangibles can be recognized in the current cursors
    def check_for_known_triangles(self, *args):
        try:
            triangles_on_screen = args[0]
            for marker in self.dict_of_markers:
                self.dict_of_markers[marker].compare_marker_triangles_and_external_triangles(triangles_on_screen)
        except Exception, e:
            print("Failure: %s" % e)
            print("Type marker:", type(marker))
            print("Marker:", marker)
            print("trianglesOnScreen type:", type(triangles_on_screen))
            print("trianglesOnScreen:", triangles_on_screen)
            print('Check checkForKnownTriangles')


    # function to return the IDs of all recognized tangibles
    def get_recognized_markers_IDs(self):
        ids = []
        for marker_id in self.dict_of_markers:
            if (self.dict_of_markers[marker_id].currently_recognized == True):
                ids = ids + [marker_id]
        return ids
        
    # function to return the recognized markers
    def get_recognized_markers(self):
        recognized_markers = []
        for marker in self.dict_of_markers.values():
            if (marker.currently_recognized == True):
                recognized_markers.append(marker)
        return recognized_markers
        
    # function returning all cursors that are part of the recognized tangibles
    def get_recognized_cursors(self):
        recognized_cursors = []
        for marker in self.dict_of_markers.values():
            # get cursor IDs from tangible
            recognized_cursors = recognized_cursors + marker.get_live_cursor_IDs()
        return recognizedCursors
        
    # register the getAllCursorsInBuffer function
    def register_cursor_provider(self, cursor_provider):
        self.get_all_cursors_in_buffer = cursor_provider
        
    # function to register a new tangible with a certain id
    def register_new_marker(self, ID):
        marker = Marker()
        marker.register_cursors(self.get_all_cursors_in_buffer(), ID=ID)
        self.dict_of_markers[ID] = marker
        print ('Marker added, ID:', ID)
        
    # function to delete a tangible with a certain id
    def delete_marker(self, ID):
        try:
            del self.dict_of_markers[ID]
            print('Marker deleted, ID:', ID)
        except:
            print("Deletion of marker failed, no marker with ID", ID)

    # function to delete all tangibles (not deleted from hard disk!)
    def delete_all_markers(self):
        self.dict_of_markers.clear()
        print('All markers deleted')

    # function to save all markers in memory to disk
    def save_markers_to_disk(self):
        try:
            file = open("markers.db", "wb") # wb = write mode
            pickle.dump(self.dict_of_markers, file)
            file.close()
            print("Markers written to disk")

        except Exception, e:
            print("Failure: %s" % e)
            print('Could not save markers to disk')
            
    # function to load markers from disk to memory
    def load_markers_from_disk(self):
        try:
            file = open("markers.db", "rb") # read mode
            self.dict_of_markers = pickle.load(file)
            print ('Markers loaded from disk')
        
        except Exception, e:
            print ("Failure: %s" % e)
            print ('Loading tangibles from file failed.')

# create a container object that can be imported
markers = MarkerContainer()

In [13]:
d = {}

In [14]:
d['a'] = 1

In [15]:
d['b'] = 2

In [16]:
d

{'a': 1, 'b': 2}

In [20]:
for elem in d.values():
    print(elem)

1
2
